In [1]:
import json
import os
import sys
from tempfile import mkdtemp
import time

import numpy as np
import matplotlib.pyplot as plt

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp
from tbd_eeg.tbd_eeg.data_analysis.Utilities.utilities import get_stim_events, find_nearest_ind

In [4]:
%matplotlib notebook

# Fix continuous file (remove extra channel)

#### Load EEGexp object

In [5]:
## this is the broken one ##
rec1 = r'P:\mouse689239\NOT_Finished_Sorting_urethane_2023-08-11_11-33-46\experiment1\recording1'
exp1 = EEGexp(rec1, preprocess=False, make_stim_csv=False)

Experiment type: electrical and sensory stimulation


## AP data

In [6]:
fixprobe = 'probeF'

# filepaths
ap_raw = exp1.ephys_params[fixprobe]['ap_continuous']
ap_time_path = exp1.ephys_params[fixprobe]['ap_timestamps']
numCh = exp1.ephys_params[fixprobe]['num_chs']

## memmap AP data
aptimestamps = np.load(ap_time_path)
rawDatamm = np.memmap(ap_raw, dtype='int16', mode='r', shape=(len(aptimestamps), 385)) # r+ will allow the file to be changed
print(rawDatamm.shape)

(158410356, 385)


Create a new file

In [7]:
print(ap_raw)
strind = ap_raw.find('.dat')
new_file = ap_raw[:strind] + '_edit' + ap_raw[strind:]
print(new_file)

P:\mouse689239\NOT_Finished_Sorting_urethane_2023-08-11_11-33-46\experiment1\probeF_sorted\continuous\Neuropix-PXI-100.0\continuous.dat
P:\mouse689239\NOT_Finished_Sorting_urethane_2023-08-11_11-33-46\experiment1\probeF_sorted\continuous\Neuropix-PXI-100.0\continuous_edit.dat


In [8]:
st1 = time.time()
fixdata = np.memmap(new_file, dtype='int16', mode='w+', shape=(rawDatamm.shape[0], numCh))
print(fixdata.shape)
e1 = time.time()
print('Time = {:.2f} min'.format((e1-st1)/60))
# This took 20 min for AP

(158410356, 384)
Time = 19.58 min


In [9]:
print(fixdata[:3,:3])

[[0 0 0]
 [0 0 0]
 [0 0 0]]


Add the data to it...unfortunately, this does require loading the whole file into memory

In [10]:
st2 = time.time()
fixdata[:,:] = rawDatamm[:,:numCh]
e2 = time.time()
print('Time = {:.2f} min'.format((e2-st2)/60))
# This took 80 min for AP and basically maxed out memory

Time = 51.30 min


In [11]:
print(rawDatamm[:3,:3])
print(fixdata[:3,:3])

[[ 56 204  31]
 [ 56 144  91]
 [ 92  83  31]]
[[ 56 204  31]
 [ 56 144  91]
 [ 92  83  31]]


In [12]:
print(rawDatamm[-3:,-3:])
print(fixdata[-3:,-3:])

[[-11   4   0]
 [ 49  28   0]
 [-11   4   0]]
[[-106  -11    4]
 [ -34   49   28]
 [ -58  -11    4]]


Write data to new file

In [13]:
st3 = time.time()
fixdata.flush()
e3 = time.time()
print('Time = {:.2f} min'.format((e3-st3)/60))
# This took 4 min for AP

Time = 19.30 min


In [14]:
del rawDatamm
del fixdata

Check it

In [15]:
fixeddata = np.memmap(new_file, dtype='int16', mode='r', shape=(len(aptimestamps), numCh))
print(fixeddata[:3,:3])

[[ 56 204  31]
 [ 56 144  91]
 [ 92  83  31]]


## LFP data

In [6]:
fixprobe = 'probeF'

# filepaths
lfp_raw = exp1.ephys_params[fixprobe]['lfp_continuous']
lfp_time_path = exp1.ephys_params[fixprobe]['lfp_timestamps']
numCh = exp1.ephys_params[fixprobe]['num_chs']

## memmap LFP data
lfptimestamps = np.load(lfp_time_path)
rawLFPmm = np.memmap(lfp_raw, dtype='int16', mode='r', shape=(len(lfptimestamps), 385))
print(rawLFPmm.shape)

(13200863, 385)


Create a new file

In [7]:
## For LFP ##
print(lfp_raw)
strind = lfp_raw.find('.dat')
new_file = lfp_raw[:strind] + '_edit' + lfp_raw[strind:]
print(new_file)

P:\mouse689239\NOT_Finished_Sorting_urethane_2023-08-11_11-33-46\experiment1\probeF_sorted\continuous\Neuropix-PXI-100.1\continuous.dat
P:\mouse689239\NOT_Finished_Sorting_urethane_2023-08-11_11-33-46\experiment1\probeF_sorted\continuous\Neuropix-PXI-100.1\continuous_edit.dat


In [8]:
st1 = time.time()
fixdata = np.memmap(new_file, dtype='int16', mode='w+', shape=(rawLFPmm.shape[0], numCh))
print(fixdata.shape)
e1 = time.time()
print('Time = {:.2f} min'.format((e1-st1)/60))
# This took 2 min for LFP

(13200863, 384)
Time = 1.63 min


In [9]:
print(fixdata[:3,:3])

[[0 0 0]
 [0 0 0]
 [0 0 0]]


Add data to it

In [10]:
st2 = time.time()
fixdata[:,:] = rawLFPmm[:,:numCh]
e2 = time.time()
print('Time = {:.2f} min'.format((e2-st2)/60))
# This took 4 min for LFP

Time = 4.48 min


In [13]:
print(rawLFPmm[:3,:3])

[[101 -11  -8]
 [125  13  40]
 [149  85 112]]


In [14]:
print(fixdata[:3,:3])

[[101 -11  -8]
 [125  13  40]
 [149  85 112]]


Write data to new file

In [15]:
st3 = time.time()
fixdata.flush()
e3 = time.time()
print('Time = {:.2f} min'.format((e3-st3)/60))
# This took 2 min for LFP

Time = 1.48 min


In [16]:
del rawLFPmm
del fixdata

Check it

In [21]:
fixeddata = np.memmap(new_file, dtype='int16', mode='r', shape=(len(lfptimestamps), numCh))
print(fixeddata[:3,:3])

[[  98   21 -116]
 [  74   45  -92]
 [  98   93  -92]]


# Test editing/writing dat files

In [5]:
test_dir = r"C:\Users\lesliec\Documents\test_dat_file"

In [6]:
testfilename = os.path.join(test_dir, 'test_write_file_4.dat')
copyfilename = os.path.join(test_dir, 'test_write_file_4copy1.dat')

Create fake data

In [7]:
datasize = (500, 33)

In [8]:
data = np.arange(datasize[0] * datasize[1], dtype='int16')
data.resize(datasize)
print(data.shape)

(500, 33)


Write a .dat file

In [9]:
fp = np.memmap(os.path.join(mkdtemp(), testfilename), dtype='int16', mode='w+', shape=datasize)
print(fp[:5,:5])

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [10]:
fp[:] = data[:]

In [11]:
fp.flush()

Test that it worked

In [12]:
newfp = np.memmap(testfilename, dtype='int16', mode='r')
print(newfp.shape)

(16500,)


In [13]:
newfp

memmap([    0,     1,     2, ..., 16497, 16498, 16499], dtype=int16)

In [14]:
del newfp

Load test file and copy part into new file

In [15]:
rawdata = np.memmap(testfilename, dtype='int16', mode='r', shape=datasize)

In [16]:
print(rawdata[:5, :5])
print(rawdata.shape)
print(type(rawdata))

[[  0   1   2   3   4]
 [ 33  34  35  36  37]
 [ 66  67  68  69  70]
 [ 99 100 101 102 103]
 [132 133 134 135 136]]
(500, 33)
<class 'numpy.memmap'>


In [17]:
# copydata = np.memmap(os.path.join(mkdtemp(), copyfilename), dtype='int16', mode='w+', shape=(rawdata.shape[0],rawdata.shape[1]-1))
copydata = np.memmap(copyfilename, dtype='int16', mode='w+', shape=(rawdata.shape[0],rawdata.shape[1]-1))
# copydata = np.memmap(copyfilename, dtype='int16', mode='r+', shape=(rawdata.shape[0],rawdata.shape[1]-1))

In [18]:
print(copydata[:5, :5])

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [19]:
copydata[:,:] = rawdata[:,:-1]
## copydata = rawdata[:,:-1] DOES NOT WORK!!!

In [20]:
print(copydata[:5, :5])
print(copydata.shape)
print(type(copydata))

[[  0   1   2   3   4]
 [ 33  34  35  36  37]
 [ 66  67  68  69  70]
 [ 99 100 101 102 103]
 [132 133 134 135 136]]
(500, 32)
<class 'numpy.memmap'>


In [21]:
copydata.flush()
# del copydata

#### Test it

In [7]:
tcdata = np.memmap(copyfilename, dtype='int16', mode='r')
print(tcdata.shape)
print(tcdata)

(16000,)
[    0     1     2 ... 16496 16497 16498]


#### Try deleting values

# Testing weird continuous.dat files

#### Load EEGexp object

In [5]:
## this is the broken one ##
rec1 = r'P:\mouse689239\NOT_Finished_Sorting_urethane_2023-08-11_11-33-46\experiment1\recording1'
exp1 = EEGexp(rec1, preprocess=False, make_stim_csv=False)

Experiment type: electrical and sensory stimulation


In [6]:
exp1.experiment_data

['probeB_sorted',
 'probeC_sorted',
 'probeD_sorted',
 'probeF_sorted',
 'recording1']

In [7]:
for datatype in exp1.experiment_data:
    if 'probe' in datatype:
        print(datatype[:6])
        # make paths
        ap_raw = exp1.ephys_params[datatype[:6]]['ap_continuous']
        ap_time_path = exp1.ephys_params[datatype[:6]]['ap_timestamps']
        # get raw data length
        rawDatamm = np.memmap(ap_raw, dtype='int16', mode='r') # r+ will allow the file to be changed
        timestamps = np.load(ap_time_path)
        print(' data length = {:d}'.format(len(rawDatamm)))
        print(' timestamp length = {:d}'.format(len(timestamps)))
        print(' extra samples = {:d}'.format(len(rawDatamm)%384))
        print('')

probeB
 data length = 60988010160
 timestamp length = 158410416
 extra samples = 48

probeC
 data length = 60988125660
 timestamp length = 158410716
 extra samples = 348

probeD
 data length = 60988005540
 timestamp length = 158410404
 extra samples = 36

probeF
 data length = 60987987060
 timestamp length = 158410356
 extra samples = 372



In [ ]:
exp1.ephys_params['probeB']['num_chs']

In [ ]:
60988005540 / 158410404

Plot some data at beginning and end.

Why are there 385 chs?? Ch 385 is empty, need to remove from AP and probably LFP, too.

In [ ]:
testprobe = 'probeF'
# filepaths
ap_raw = exp1.ephys_params[testprobe]['ap_continuous']
ap_time_path = exp1.ephys_params[testprobe]['ap_timestamps']
# get raw data length
rawDatamm = np.memmap(ap_raw, dtype='int16', mode='r') # r+ will allow the file to be changed
timestamps = np.load(ap_time_path)

In [ ]:
numCh = 385
# datamm = np.reshape(rawDatamm, (int(rawDatamm.size/numCh), numCh))
datamm = np.reshape(rawDatamm, (numCh, int(rawDatamm.size/numCh)), order='F')

In [ ]:
print(datamm.shape)
print(timestamps.shape)

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
ax.plot(datamm[-2:, :100].T)